# Cuaderno 12: Problema del bin packing

En el problema del *bin packing* (*empacamiento de recipientes*), se tienen dados $n$ objetos, con pesos $w_1, \ldots, w_n$. Se quiere empacar estos objetos empleando la menor cantidad de recipientes de capacidad $b$, de tal forma que los objetos empacados en un recipiente no superen nunca la capacidad de este.

Si suponemos que $I$ es el conjunto de objetos y $J$ el conjunto de recipientes. Utilizaremos dos familias de variables de decisión binarias: 

1. Variables $x_{ij}$, con $i \in I$ y $j \in J$, para indicar si el objeto $i$ es empacado en el recipiente $j$. 
2. Variables $y_j$, con $j \in J$, para indicar si el recipiente $j$ es utilizado en la solución.

El problema puede formularse como el siguiente programa lineal entero:


\begin{align*}
&\min \sum_{j \in J} y_j \\ 
&\mbox{s.r.}\\
& \sum_{j \in J} x_{ij} = 1, \quad \forall i \in I, \\
& \sum_{i \in I} w_i x_{ij} \leq b y_j, \quad \forall j \in J, \\
& x_{ij} \in \{0, 1\}, \quad \forall i \in I, j \in J,\\
& y_{j} \in \{0, 1\}, \quad \forall j \in J.
\end{align*}

Vamos a formular este modelo empleando el API Python de Gurobi. Empezamos por definir los conjuntos de índices, que en este caso serán del tipo `tuplelist`. Para usar este tipo es necesario importar primero el módulo de Gurobi:

In [11]:
from gurobipy import *
n = 5
I = tuplelist(range(1, n+1)) # indexamos los objetos como {1, ..., n}
J = tuplelist(range(1, n+1)) # indexamos los recipientes como {1, ..., n}

print ('I:= {}'.format(I))
print ('J:= {}'.format(J))

I:= [1, 2, 3, 4, 5]
J:= [1, 2, 3, 4, 5]


Luego definimos los parámetros del modelo. Aquellos parámetros que dependen de índices se definen como diccionarios:

In [12]:
# los pesos están dados por un diccionario cuyas claves son elementos de I
w = tupledict({ 1 : 50, 
      2 : 45, 
      3 : 55, 
      4 : 40, 
      5 : 48})
b = 100 # capacidad de los recipientes

El siguiente paso es crear el objeto modelo:

In [13]:
m = Model('bin-packing')

Para crear las variables, podemos usar el método `addVars`. Este método permite crear una familia de variables indexadas por uno o más objetos del tipo `tuplelist`. En nuestro caso, las variables $x_{ij}$ están indexadas por todas las combinaciones de objetos $i \in I$ y recipientes $j \in J$. Por lo tanto, los dos primeros argumentos de la función serán `I` y `J`:

In [14]:
# creamos variables x indexadas por I x J
x = m.addVars(I, J, vtype = GRB.BINARY, name="x")

Los demás argumentos de la función son los mismos que los argumentos de `addVar`. Recordar que el argumento `name` en este caso no fija el nombre de una variable, sino un nombre genérico que se utilizará como prefijo para el nombre de cada variable individual, y que será completado con los valores de los índices.

Notar que la función `addVars` retorna un objeto del tipo `tupledict` con todas las variables:

In [16]:
# x es del tipo tupledict
print (type(x))
# antes de consultar el contenido de cualquier variable, es necesario llamar a update()
m.update()
print (x)
# las claves de x forman un tuplelist
print (x.keys())
# para acceder a un elemento, hay que usar su tupla como índice; los paréntesis pueden obviarse
# "x" se usa como prefijo para el nombre de la variable
print(x[(1,2)].varName)
# recordar que los paréntesis alrededor de una tupla pueden omitirse si no hay riesgo de confusión:
print(x[5,4].varName)

<class 'gurobipy.tupledict'>
{(1, 1): <gurobi.Var x[1,1]>, (1, 2): <gurobi.Var x[1,2]>, (1, 3): <gurobi.Var x[1,3]>, (1, 4): <gurobi.Var x[1,4]>, (1, 5): <gurobi.Var x[1,5]>, (2, 1): <gurobi.Var x[2,1]>, (2, 2): <gurobi.Var x[2,2]>, (2, 3): <gurobi.Var x[2,3]>, (2, 4): <gurobi.Var x[2,4]>, (2, 5): <gurobi.Var x[2,5]>, (3, 1): <gurobi.Var x[3,1]>, (3, 2): <gurobi.Var x[3,2]>, (3, 3): <gurobi.Var x[3,3]>, (3, 4): <gurobi.Var x[3,4]>, (3, 5): <gurobi.Var x[3,5]>, (4, 1): <gurobi.Var x[4,1]>, (4, 2): <gurobi.Var x[4,2]>, (4, 3): <gurobi.Var x[4,3]>, (4, 4): <gurobi.Var x[4,4]>, (4, 5): <gurobi.Var x[4,5]>, (5, 1): <gurobi.Var x[5,1]>, (5, 2): <gurobi.Var x[5,2]>, (5, 3): <gurobi.Var x[5,3]>, (5, 4): <gurobi.Var x[5,4]>, (5, 5): <gurobi.Var x[5,5]>}
<gurobi.tuplelist (25 tuples, 2 values each):
 ( 1 , 1 )
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
 ( 1 , 5 )
 ( 2 , 1 )
 ( 2 , 2 )
 ( 2 , 3 )
 ( 2 , 4 )
 ( 2 , 5 )
 ( 3 , 1 )
 ( 3 , 2 )
 ( 3 , 3 )
 ( 3 , 4 )
 ( 3 , 5 )
 ( 4 , 1 )
 ( 4 , 2 )
 ( 4 , 3 )
 

De manera similar, creamos las variables $y_j$ indexadas por J:

In [17]:
y = m.addVars(J, vtype = GRB.BINARY, name="y")

Para construir la función objetivo, podemos usar el método `sum` de la clase `tupledict`:

In [18]:
# la función objetivo es la suma de las variables
m.setObjective(y.sum('*'), GRB.MINIMIZE)

Definimos ahora las restricciones:

1. Cada objeto es empacado en un recipiente: para expresar esta restricción, podemos emplear el método `sum` de la clase `tupledict`, con un criterio de selección `(i, '*')`. Notar que hay una restricción para cada $i \in I$, por lo que empleamos el método `addConstrs` con un generador que depende de `i`:

In [19]:
# Cada objeto es asignado a un recipiente
m.addConstrs((x.sum(i, '*') == 1 for i in I), "asig") 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

2. Se asignan objetos únicamente a recipientes utilizados, y respetando su capacidad: En este caso utilizamos la función `quicksum`, conjuntamente con un generador que depende de `i`, para construir la expresión $\sum_i w_i x_{ij}$. Notar además que hay una restricción para cada $j \in J$, por lo que empleamos el método `addConstrs` y anidamos la expresión generadora anterior en una nueva expresión que depende de `j`:

In [20]:
# Se asignan objetos únicamente a recipientes utilizados, y respetando su capacidad
m.addConstrs((quicksum(w[i] * x[i, j] for i in I) <= b * y[j] for j in J), "capac")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

Finalmente, optimizamos el modelo:

In [21]:
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 10 rows, 30 columns and 55 nonzeros
Model fingerprint: 0x893a42ca
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve time: 0.03s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 0 continuous, 30 integer (30 binary)

Root relaxation: objective 2.450000e+00, 15 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.45000    0    1    4.00000    2.45000  38.7%     -    0s
H    0     0                       3.0000000    2.45000  18.3%     -    0s
     0     0    2.45000    0    1    3.00000    2.45000  18.3%     -    0s

Explored 1 nodes (15 simplex ite

Terminada la optimización, podemos mostrar las soluciones. Notar que se usa la sintaxis `x[i, j]` para acceder a la variable $x_{ij}$, pues `x` es un diccionario cuyas claves son pares ordenados del conjunto $I \times J$. Recordar que la propiedad `x` de un objeto variable nos permite acceder a su valor en la solución, indistintamente del nombre del objeto.

Notar que al comparar si los valores de las variables binarias son iguales a 1 usamos la condición `>=0.99 ` en lugar de la condición `== 1`, pese que se trata de variables enteras. El motivo es que Gurobi (al igual que otros solvers) trabaja internamente con valores de punto flotante (decimales) para todas las variables del modelo, debido a que debe resolver relajaciones lineales del programa entero. Gurobi asume que una variable tiene un valor entero cuando su valor está dentro de un rango de tolerancia del entero más próximo. Este rango está fijado en el parámetro `IntFeasTol` y su valor por defecto es $10^{-5}$. Más información puede consultarse en [este artículo](https://support.gurobi.com/hc/en-us/articles/360012237872-Why-does-Gurobi-sometimes-return-values-for-integer-variables-that-are-not-integers-).

In [22]:
for j in J:
    # y[j].x es el valor de la variable y_j en la solución
    if y[j].x >= 0.99:
        print("Se usó el recipiente {}.".format(j))
        
for i in I:
    for j in J:
        # y[j].x es el valor de la variable y_j en la solución
        if x[i, j].x >= 0.99:
            print("Objeto {} asignado al recipiente {}.".format(i, j))

Se usó el recipiente 1.
Se usó el recipiente 4.
Se usó el recipiente 5.
Objeto 1 asignado al recipiente 5.
Objeto 2 asignado al recipiente 1.
Objeto 3 asignado al recipiente 4.
Objeto 4 asignado al recipiente 4.
Objeto 5 asignado al recipiente 5.


## Código completo

Reproducimos a continuación el código completo del ejemplo:

In [26]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de bin packing
# EPN (2019)

from gurobipy import *
try:
    n = 5
    I = tuplelist(range(1, n+1)) # indexamos los objetos como {1, ..., n}
    J = tuplelist(range(1, n+1)) # indexamos los recipientes como {1, ..., n}

    # los pesos están dados por un diccionario cuyas claves son elementos de I
    w = { 1 : 50, 
          2 : 45, 
          3 : 55, 
          4 : 40, 
          5 : 48}
    b = 100 # capacidad de los recipientes

    m = Model('bin-packing')

    # creamos variables x indexadas por I x J
    x = m.addVars(I, J, vtype = GRB.BINARY, name="x")

    # creamos variables y indexadas por J
    y = m.addVars(J, vtype = GRB.BINARY, name="y")

    # la función objetivo es la suma de las variables
    m.setObjective(y.sum('*'), GRB.MINIMIZE)

    # Restricciones
    # Cada objeto es asignado a un recipiente
    m.addConstrs((x.sum(i, '*') == 1 for i in I), "asig") 

    # Se asignan objetos únicamente a recipientes utilizados, y respetando su capacidad
    m.addConstrs((quicksum(w[i] * x[i, j] for i in I) <= b * y[j] for j in J), "capac")
    
    # Resolver el modelo
    m.optimize()

    # Mostrar la solución
    for j in J:
        if y[j].x >= 0.99:
            print("Se usó el recipiente {}.".format(j))
        
    for i in I:
        for j in J:
            if x[i, j].x >= 0.99:
                print("Objeto {} asignado al recipiente {}.".format(i, j))

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 10 rows, 30 columns and 55 nonzeros
Model fingerprint: 0x893a42ca
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve time: 0.00s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 0 continuous, 30 integer (30 binary)

Root relaxation: objective 2.450000e+00, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.45000    0    1    4.00000    2.45000  38.7%     -    0s
H    0     0                       3.0000000    2.45000  18.3%     -    0s
     0     0    2.45000    0    1    3.00000    2.45000  18.3%     -    0s

Explored 1 nodes (15 simplex ite